In [1]:
import tensorflow as tf
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
%matplotlib inline

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration( gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20)])

In [3]:
tf.__version__

'2.0.1'

In [4]:
tf.test.is_gpu_available()

True

### Split the dataset

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()  #50000 train, 10000 test

In [6]:
category_index=[]     # get a list of turples, each turple is the set of a class i.e. total_set[class_nb][img/label]

for i in range(10):
    index0 = np.where(y_train == i)
    index = index0[0]
    np.random.shuffle(index)
    category_index.append(index)

### Assign data to each worker_node

In [7]:
# '''
# Scenario iid.10w.5000
# Here get 10 worker nodes.
# worker: 10 class * 50 class_data  =500
# '''
# index_list=[]
# worker_nb=10
# class_nb=10
# class_data=50
# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][j*class_data:j*class_data+class_data] for i in range(class_nb)])
#     index_list.append(index)
# len(index_list)

In [9]:
# '''
# Scenario 5c.10w.5000
# Here get 10 worker nodes.
# worker: 5 class * 100 class_data  =500

# '''
# index_list=[]
# worker_nb=10
# class_nb=5
# class_data=100

# class_set=[]
# for i in range(worker_nb):
#     x=[i+j for j in range(class_nb)]
#     for c in range(len(x)):
#         while True:
#             if x[c]<10:
#                 break
#             x[c]=x[c]-10  
#     class_set.append(x)
# print(class_set)

# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][j*class_data:j*class_data+class_data] for i in class_set[j]])
#     index_list.append(index)
# len(index_list)


[[0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3], [0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3], [0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3], [0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3], [0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3], [0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 

100

In [17]:
# '''
# Scenario 2c.10w.5000
# Here get 10 worker nodes.
# worker: 2 class * 250 class_data  =500

# '''
# index_list=[]
# worker_nb=10
# class_nb=2
# class_data=250

# class_set=[]
# for i in range(worker_nb):
#     x=[i+j for j in range(class_nb)]
#     for c in range(len(x)):
#         while True:
#             if x[c]<10:
#                 break
#             x[c]=x[c]-10  
#     class_set.append(x)
# print(class_set)

# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][j*class_data:j*class_data+class_data] for i in class_set[j]])
#     index_list.append(index)
# len(index_list)

[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 0], [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 0], [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 0], [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 0]]


40

In [18]:
# '''
# Scenario 1c.10w.5000
# Here get 10 worker nodes.
# worker: 1 class * 500 class_data  =500

# '''
# index_list=[]
# worker_nb=10
# class_nb=1
# class_data=500
# for i in range(worker_nb):
#     n=i%10
#     k=i//10
# #     print(n,k)
#     index=category_index[n][class_data*k:class_data*k+class_data]
#     index_list.append(index)

In [19]:
# '''
# Scenario Union-1c
# A. 1c.100.40w.4000
# B. 10c.10.10w.1000
# '''
# #A
# index_list=[]
# worker_nb=40
# class_nb=1
# class_data=100
# for i in range(worker_nb):
#     n=i%10
#     k=i//10
# #     print(n,k)
#     index=category_index[n][class_data*k:class_data*k+class_data]
#     start_point=class_data*k+class_data
#     index_list.append(index)
    
# print(start_point)


# #B
# worker_nb=10
# class_nb=10
# class_data=10
# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][start_point+j*class_data:start_point+j*class_data+class_data] for i in range(class_nb)])
#     end_point=start_point+j*class_data+class_data
#     index_list.append(index)
# print(end_point)
# worker_nb=len(index_list)



In [20]:
# '''
# Scenario Union-2c
# A. 2c.100.40w.8000
# B. 10c.10.10w.1000
# '''
# #A
# index_list=[]
# worker_nb=40
# class_nb=2
# class_data=100

# class_set=[]
# for i in range(worker_nb):
#     x=[i+j for j in range(class_nb)]
#     for c in range(len(x)):
#         while True:
#             if x[c]<10:
#                 break
#             x[c]=x[c]-10  
#     class_set.append(x)
# print(class_set)

# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][j*class_data:j*class_data+class_data] for i in class_set[j]])
#     index_list.append(index)

# #B
# start_point=800
# worker_nb=10
# class_nb=10
# class_data=10
# for j in range(worker_nb):
#     index=np.concatenate([category_index[i][start_point+j*class_data:start_point+j*class_data+class_data] for i in range(class_nb)])
#     end_point=start_point+j*class_data+class_data
#     index_list.append(index)
# print(end_point)
# worker_nb=len(index_list)

In [21]:
len(index_list)

40

In [22]:
index1=index_list[0]
np.unique(y_train[index1],return_counts=True)

(array([0, 1], dtype=uint8), array([50, 50], dtype=int64))

### Creat & save index to each worker_node

In [ ]:
base_path='worker_nodes'
workers=['model'+str(int(i+1)) for i in range(worker_nb)] #save like 'worker_nodes/model1/index.npy'
shutil.rmtree(base_path, ignore_errors=True)
for i in range(worker_nb):
    worker=workers[i]
    worker_dir=os.path.join(base_path,worker)
    try:
        os.makedirs(worker_dir)
    except:
        pass
    file_path = os.path.join(worker_dir,'index.npy') ## TODO: copy the training code of the worker to this dir
    np.save(file_path, index_list[i])

### Build model & save

In [ ]:
index_collection=glob.glob('worker_nodes/*/index.npy')
central_weight_path=os.path.join('central_node','ANN_model.h5')

In [ ]:
from models import ANN_model

model=ANN_model()
shutil.rmtree('central_node', ignore_errors=True)
os.makedirs('central_node')
model.save_weights(central_weight_path)